In [2]:
import argparse
import datetime
import os
import re
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default
import time

import numpy as np
import tensorflow as tf

from cifar10 import CIFAR10

from cifar_competition_model_class import Model as Model_CNN


# TODO: Define reasonable defaults and optionally more parameters
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=10, type=int, help="Number of epochs.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=4, type=int, help="Maximum number of threads to use.")
parser.add_argument("--cnn", default='CB-32-3-2-same,M-2-1,CB-64-3-2-same,M-2-1,R-[CB-64-3-1-same,CB-64-3-1-same],CB-126-3-2-same,M-2-1,F,H-2048,H-2048', type=str, help="CNN architecture.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.9, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--decay_steps", default=100, type=int, help="decay_steps.")
parser.add_argument("--vgg_rep", default=100, type=int, help="number of repetitions of conv/maxpool in the style of vgg")

def main(args: argparse.Namespace): # -> None:
    # Fix random seeds and threads
    tf.keras.utils.set_random_seed(args.seed)
    tf.config.threading.set_inter_op_parallelism_threads(args.threads)
    tf.config.threading.set_intra_op_parallelism_threads(args.threads)

    # Create logdir name
    # args.logdir = os.path.join("logs", "{}-{}-{}".format(
    #     os.path.basename(globals().get("__file__", "notebook")),
    #     datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
    #     ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
    # ))

    args.logdir = os.path.join("logs", "{}-{}".format(
        os.path.basename(globals().get("__file__", "notebook")),
        datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
        # ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
    ))

    pars = "{}".format(
        ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
    )

    # Load data
    cifar = CIFAR10()

    # TODO: Create the model and train it
    model = Model_CNN(args)


    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                            rotation_range=20,  zoom_range=0.2, width_shift_range=0.1, horizontal_flip=True
                            )
    # tb_callback = tf.keras.callbacks.TensorBoard(args.logdir[:-57], update_freq=100, profile_batch=0)
    tb_callback = tf.keras.callbacks.TensorBoard(args.logdir[:-56], update_freq=100, profile_batch=0)
    tb_callback._close_writers = lambda: None # A hack allowing to keep the writers open.
    model.fit(
        train_generator.flow(cifar.train.data["images"], 
                            y=cifar.train.data["labels"], 
                            batch_size=args.batch_size, 
                            seed=args.seed, 
                            shuffle=True),
        shuffle=False, 
        batch_size=args.batch_size, epochs=args.epochs,
        validation_data=(cifar.dev.data["images"], cifar.dev.data["labels"]),
        callbacks=[tb_callback]
    )

    # np.savetxt(args.logdir+os.sep+'pars.txt', pars)

    # Generate test set annotations, but in `args.logdir` to allow parallel execution.
    os.makedirs(args.logdir, exist_ok=True)
    with open(os.path.join(args.logdir, "cifar_competition_test.txt"), "w", encoding="utf-8") as predictions_file:
        for probs in model.predict(cifar.test.data["images"], batch_size=args.batch_size):
            print(np.argmax(probs), file=predictions_file)
    
    with open(os.path.join(args.logdir, "pars.txt"), "w", encoding="utf-8") as pars_file:
        print(pars, file=pars_file)




In [3]:
# TODO: Define reasonable defaults and optionally more parameters
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=150, type=int, help="Number of epochs.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=4, type=int, help="Maximum number of threads to use.")
parser.add_argument("--cnn", default='CB-64-3-2-same,M-2-1,CB-128-3-2-same,M-2-1,R-[CB-64-3-1-same,CB-128-3-1-same],CB-256-3-2-same,M-2-1,F,H-4096,H-4096', type=str, help="CNN architecture.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.9, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--decay_steps", default=100, type=int, help="decay_steps.")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)

Namespace(batch_size=256, cnn='CB-64-3-2-same,M-2-1,CB-128-3-2-same,M-2-1,R-[CB-64-3-1-same,CB-128-3-1-same],CB-256-3-2-same,M-2-1,F,H-4096,H-4096', decay_steps=100, epochs=150, learning_rate=0.01, learning_rate_final=0.001, momentum=0.9, optimizer='SGD', seed=42, threads=4)


In [5]:
# time.sleep(900)

In [6]:
main(args)

Epoch 1/150
176/176 [==============================] - 29s 103ms/step - loss: 1.5832 - accuracy: 0.4364 - val_loss: 3.2348 - val_accuracy: 0.1052
Epoch 2/150
176/176 [==============================] - 17s 97ms/step - loss: 1.3197 - accuracy: 0.5263 - val_loss: 2.8829 - val_accuracy: 0.2132
Epoch 3/150
176/176 [==============================] - 17s 98ms/step - loss: 1.2475 - accuracy: 0.5536 - val_loss: 1.6779 - val_accuracy: 0.4038
Epoch 4/150
176/176 [==============================] - 17s 97ms/step - loss: 1.1823 - accuracy: 0.5790 - val_loss: 1.1754 - val_accuracy: 0.5762
Epoch 5/150
176/176 [==============================] - 18s 100ms/step - loss: 1.1332 - accuracy: 0.5978 - val_loss: 1.1303 - val_accuracy: 0.5960
Epoch 6/150
176/176 [==============================] - 18s 99ms/step - loss: 1.0931 - accuracy: 0.6128 - val_loss: 1.0647 - val_accuracy: 0.6176
Epoch 7/150
176/176 [==============================] - 18s 103ms/step - loss: 1.0507 - accuracy: 0.6270 - val_loss: 1.2638 - val

In [ ]:
time.sleep(10)

In [ ]:
# TODO: Define reasonable defaults and optionally more parameters
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=250, type=int, help="Number of epochs.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=4, type=int, help="Maximum number of threads to use.")
parser.add_argument("--cnn", default='CB-64-3-2-same,M-2-1,CB-128-3-2-same,M-2-1,R-[CB-64-3-1-same,CB-128-3-1-same],CB-256-3-2-same,M-2-1,F,H-4096,H-4096', type=str, help="CNN architecture.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.9, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--decay_steps", default=100, type=int, help="decay_steps.")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)

In [ ]:
main(args)

In [ ]:
time.sleep(3600)